# NYC Schools Analysis using SQL and Python

This notebook connects to a PostgreSQL database hosted on Neon that contains
NYC high school data. Using SQL and Python (pandas), we will:

- Connect to the database using SQLAlchemy
- Explore the available tables and their schemas
- Answer key analytical questions:
  1. How many schools are there in each borough?
  2. What is the average percentage of English Language Learners (ELL) per borough?
  3. Which are the top 3 schools in each borough with the highest percentage of
     special education students (sped_percent)?
- Summarize key insights in Markdown cells.


In [14]:
# 1. Imports

import pandas as pd
from sqlalchemy import create_engine

## 2. Database Connection

We connect to the PostgreSQL database using SQLAlchemy.

In [15]:
# 2. Database Connection Setup (Neon PostgreSQL)

DATABASE_URL = (
    "postgresql+psycopg2://neondb_owner:a9Am7Yy5r9_T7h4OF2GN"
    "@ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech:5432/neondb"
    "?sslmode=require"
)

# Create the SQLAlchemy engine
engine = create_engine(DATABASE_URL)

# Quick test query
test_query = "SELECT 1 AS test_column;"
test_df = pd.read_sql(test_query, engine)
test_df

,test_column
0,1


## 3. Helper Function for Running SQL Queries

To keep the notebook clean and reusable, we define a small helper function
that sends a SQL query to the database and returns the result as a pandas DataFrame.

In [16]:
# 3. Helper Function

def run_query(sql: str) -> pd.DataFrame:
    """
    Execute a SQL query using the global SQLAlchemy engine
    and return the result as a pandas DataFrame.
    """
    return pd.read_sql(sql, engine)

## 4. Explore the Database Schema

Before writing analytical queries, we first explore:
- Which tables are available
- The columns in the key tables:
  - `nyc_schools.high_school_directory`
  - `nyc_schools.school_demographics`
  - `nyc_schools.school_safety_report`

In [17]:
# 4.1 List tables in the nyc_schools schema

tables_sql = """
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema = 'nyc_schools'
ORDER BY table_name;
"""

tables_df = run_query(tables_sql)
tables_df

,table_schema,table_name
0,nyc_schools,Essam_alasaad_sat_results
1,nyc_schools,abida_sultana_sat_scores
2,nyc_schools,anastasia_sat_results
3,nyc_schools,bianca_sat_results
4,nyc_schools,darel-kigha_sat_results
5,nyc_schools,dido_sat_results
6,nyc_schools,hakim-murphy_sat_results
7,nyc_schools,heike_reichert_sat_results
8,nyc_schools,high_school_directory
9,nyc_schools,isabella_leach_sat_results


In [18]:
# 4.2 Inspect columns of high_school_directory

hs_dir_cols_sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'nyc_schools'
  AND table_name = 'high_school_directory'
ORDER BY ordinal_position;
"""

hs_dir_cols_df = run_query(hs_dir_cols_sql)
hs_dir_cols_df

,column_name,data_type
0,dbn,text
1,school_name,text
2,borough,text
3,building_code,text
4,phone_number,text
...,...,...
100,Zip Codes,character varying
101,Community Districts,character varying
102,Borough Boundaries,character varying
103,City Council Districts,character varying


In [19]:
# 4.3 Inspect columns of school_demographics

demo_cols_sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'nyc_schools'
  AND table_name = 'school_demographics'
ORDER BY ordinal_position;
"""

demo_cols_df = run_query(demo_cols_sql)
demo_cols_df

,column_name,data_type
0,dbn,character varying
1,Name,character varying
2,schoolyear,integer
3,fl_percent,character varying
4,frl_percent,real
5,total_enrollment,integer
6,prek,character varying
7,k,character varying
8,grade1,character varying
9,grade2,character varying


In [20]:
# 4.4 Inspect columns of school_safety_report

safety_cols_sql = """
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'nyc_schools'
  AND table_name = 'school_safety_report'
ORDER BY ordinal_position;
"""

safety_cols_df = run_query(safety_cols_sql)
safety_cols_df

,column_name,data_type
0,school_year,text
1,building_code,text
2,dbn,text
3,location_name,text
4,location_code,text
5,address,text
6,borough,text
7,geographical_district_code,double precision
8,register,text
9,building_name,text


## 5. Analytical Questions

In this section, we address the three main analytical questions using SQL queries
and analyze the results using pandas.

### 5.1 Question 1 – School Distribution: How many schools are there in each borough?

We want to understand how schools are distributed geographically across NYC boroughs.
To do this, we:

- Use the `nyc_schools.high_school_directory` table
- Group by `borough`
- Count the number of schools per borough

In [33]:
# 5.1 Number of schools by borough

q1_sql = """
SELECT
    borough,
    COUNT(*) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough
ORDER BY school_count DESC;
"""

q1_df = run_query(q1_sql)
q1_df

,borough,school_count
0,Brooklyn,121
1,Bronx,118
2,Manhattan,106
3,Queens,80
4,Staten Island,10


#### Interpretation (Q1)

- The table above shows the total number of schools in each borough.
- This helps identify which boroughs have the highest concentration of high schools.
You can use this as a starting point for understanding geographic distribution
and potential resource allocation across NYC.

### 5.2 – Average ELL Percentage by Borough (Latest School Year)

In this section, we compute the **average percentage of English Language Learners (ELL)**  
for each borough, using data from the **latest available school year** in the
`school_demographics` table.

Our approach is:

1. Identify the **latest available school year** in the `school_demographics` table.
2. Restrict the analysis to that school year only.
3. Join `school_demographics` with `high_school_directory` on the common school
   identifier `dbn`.
4. Group the joined data by `borough`.
5. Compute the average `ell_percent` for each borough in that latest school year.

In [38]:
school_years_sql = """
SELECT DISTINCT schoolyear
FROM nyc_schools.school_demographics
ORDER BY schoolyear;
"""

school_years_df = run_query(school_years_sql)
school_years_df

,schoolyear
0,20052006
1,20062007
2,20072008
3,20082009
4,20092010
5,20102011
6,20112012


In [35]:
# 5.2 Check the latest available school year in the demographics table
latest_year_sql = """
SELECT MAX(schoolyear) AS max_year
FROM nyc_schools.school_demographics;
"""

latest_year_df = run_query(latest_year_sql)
latest_year_df

,max_year
0,20112012


In [41]:
# Compute average ELL percentage by borough for the latest school year

q2_latest_sql = """
WITH latest_year AS (
    SELECT MAX(schoolyear) AS max_year
    FROM nyc_schools.school_demographics
)
SELECT
    d.borough,
    AVG(s.ell_percent) AS avg_ell_percent
FROM nyc_schools.school_demographics AS s
JOIN nyc_schools.high_school_directory AS d
    ON s.dbn = d.dbn
JOIN latest_year ly
    ON s.schoolyear = ly.max_year
GROUP BY d.borough
ORDER BY d.borough;
"""

q2_latest_df = run_query(q2_latest_sql)
q2_latest_df

,borough,avg_ell_percent
0,Manhattan,11.96


In [42]:
# Inspect how many records per borough are present in the latest year

debug_latest_sql = """
WITH latest_year AS (
    SELECT MAX(schoolyear) AS max_year
    FROM nyc_schools.school_demographics
)
SELECT
    d.borough,
    COUNT(*) AS row_count
FROM nyc_schools.school_demographics AS s
JOIN nyc_schools.high_school_directory AS d
    ON s.dbn = d.dbn
JOIN latest_year ly
    ON s.schoolyear = ly.max_year
GROUP BY d.borough
ORDER BY d.borough;
"""

debug_latest_df = run_query(debug_latest_sql)
debug_latest_df

,borough,row_count
0,Manhattan,5


#### Interpretation (Q2 – Latest School Year)

The query above computes the **average percentage of English Language Learners (ELL)**
by borough for the **latest school year available** in the `school_demographics` table.

From the data, the latest `school_year` is **20112012**, which corresponds to the
**2011–2012 school year**.

For that school year, the result shows:

- **Manhattan** → `avg_ell_percent ≈ 11.96`

This means that, in the 2011–2012 school year, high schools located in Manhattan
have on average about **12% English Language Learners**.

In this database snapshot, only Manhattan appears in the joined data for the latest
year. If additional boroughs were present, the same query would allow us to compare
average ELL percentages across all boroughs for that school year.

### 5.3 Question 3 – Special Education: Top 3 schools in each borough by sped_percent

We want to identify the schools that serve the highest proportion of special education
students in each borough.

We assume:
- `nyc_schools.school_demographics.sped_percent` = percentage of special education students
- `nyc_schools.school_demographics.dbn` = school identifier
- `nyc_schools.high_school_directory.dbn`, `school_name`, `borough`

Approach:
1. Join demographics and directory on `dbn`.
2. Within each borough, rank schools by `sped_percent` in descending order.
3. Keep the top 3 schools per borough.

In [27]:
# 5.3 Top 3 schools per borough by special education percentage

q3b_sql = """
WITH school_sped AS (
    SELECT
        d.borough,
        d.school_name,
        s.dbn,
        MAX(s.sped_percent) AS sped_percent
    FROM nyc_schools.school_demographics AS s
    JOIN nyc_schools.high_school_directory AS d
        ON s.dbn = d.dbn
    WHERE s.sped_percent IS NOT NULL
    GROUP BY d.borough, d.school_name, s.dbn
),
sped_ranked AS (
    SELECT
        borough,
        school_name,
        dbn,
        sped_percent,
        ROW_NUMBER() OVER (
            PARTITION BY borough
            ORDER BY sped_percent DESC
        ) AS rn
    FROM school_sped
)
SELECT
    borough,
    rn AS borough_rank,
    school_name,
    dbn,
    sped_percent
FROM sped_ranked
WHERE rn <= 3
ORDER BY borough, borough_rank;
"""

q3b_df = pd.read_sql(q3b_sql, engine)
q3b_df

,borough,borough_rank,school_name,dbn,sped_percent
0,Manhattan,1,East Side Community School,01M450,28.8
1,Manhattan,2,Marta Valle High School,01M509,25.9
2,Manhattan,3,Henry Street School for International Studies,01M292,25.1


### Interpretation (Q3) – Top 3 Schools by Special Education Percentage

The query returns, for each borough, the three schools with the highest percentage
of special education students (`sped_percent`).

In the example above (Manhattan), the top 3 schools are:

1. **East Side Community School (01M450)** – `sped_percent = 28.8`
2. **Marta Valle High School (01M509)** – `sped_percent = 25.9`
3. **Henry Street School for International Studies (01M292)** – `sped_percent = 25.1`

Methodologically, we first aggregate `sped_percent` at the school level using
`MAX(sped_percent)` across the available records for each school, and then
rank schools within each borough in descending order of this value. This ensures
that each school appears only once in the ranking and that we truly obtain the
“top 3 schools” per borough in terms of special education representation.

## 6. Summary and Insights

In this analysis, we:

1. **Connected** to a PostgreSQL database hosted on Neon using SQLAlchemy and Python.
2. **Explored the schema** of the main tables in the `nyc_schools` schema:
   - `high_school_directory`
   - `school_demographics`
   - `school_safety_report`
3. Answered three key questions:

   - **School Distribution**  
     Using `nyc_schools.high_school_directory`, we counted how many schools are
     located in each borough. This provides a basic view of how high schools
     are geographically distributed across NYC.

   - **Language Learners (ELL)**  
     Using `nyc_schools.school_demographics` joined with `nyc_schools.high_school_directory`,
     we calculated the **average percentage of English Language Learners (ELL) by borough**
     for the **latest available school year (2011–2012)**.  
     In this dataset snapshot, only Manhattan appears in the joined data for that year,
     with an average ELL percentage of approximately **11.96%**.

   - **Special Education (SPED)**  
     Using the same join between `school_demographics` and `high_school_directory`, we
     aggregated `sped_percent` at the school level (using the maximum value per school)
     and then identified the **top 3 schools in each borough** with the highest
     percentage of special education students.  
     This ranking highlights schools with a particularly high concentration of
     special education students, which may require additional resources and support.

These results provide an initial view of how student populations and specific needs
(ELL and special education) are distributed across NYC high schools in the available data.

With additional queries, this analysis could be extended to:

- Examine safety incidents by school or borough using the `school_safety_report` table.
- Explore relationships between school demographics (ELL, SPED, enrollment) and safety metrics.
- Study trends over time by comparing multiple school years, rather than focusing only on 2011–2012.